In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore') 

import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score

In [2]:
class config:
    DATA = '/kaggle/input/vner-vlsp-2021/processed_data.csv'

# `Load processed dataset`

In [3]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [4]:
dataset = import_data(config.DATA)

Memory usage of dataframe is 22.47 MB
Memory usage after optimization is: 9.10 MB
Decreased by 59.5%


In [25]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 981556 entries, 596724 to 210189
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Word        981556 non-null    category
 1   Tag         981556 non-null    category
 2   Sentence #  981556 non-null    category
dtypes: category(3)
memory usage: 9.1 MB


In [26]:
tag_counts = dataset['Tag'].value_counts()

tags_to_keep = tag_counts[tag_counts >= 2].index

dataset = dataset[dataset['Tag'].isin(tags_to_keep)]

In [27]:
X = dataset.drop('Tag', axis=1)

In [28]:
v = DictVectorizer(sparse=False)
X = v.fit_transform(X.to_dict('records'))
y = dataset.Tag.values

In [29]:
classes = np.unique(y)
classes = classes.tolist()
# classes

In [30]:
new_classes = classes.copy()
new_classes.pop()
# new_classes

'O'

In [31]:
unique, counts = np.unique(y, return_counts=True)
class_distribution = dict(zip(unique, counts))
print("Class distribution:", class_distribution)

Class distribution: {'B-DATETIME-DURATION': 2, 'B-LOCATION': 6, 'B-LOCATION-GPE': 7, 'B-LOCATION-STRUC': 3, 'B-ORGANIZATION': 4, 'B-ORGANIZATION-SPORTS': 3, 'B-PERSON': 16, 'B-PERSONTYPE': 6, 'B-PRODUCT': 3, 'B-QUANTITY': 6, 'B-QUANTITY-NUM': 6, 'B-QUANTITY-ORD': 3, 'B-QUANTITY-PER': 2, 'I-DATETIME': 10, 'I-DATETIME-DATE': 4, 'I-DATETIME-DATERANGE': 4, 'I-DATETIME-DURATION': 3, 'I-DATETIME-SET': 2, 'I-DATETIME-TIME': 3, 'I-EVENT': 2, 'I-LOCATION-GPE': 3, 'I-ORGANIZATION': 14, 'I-PERSON': 4, 'I-PERSONTYPE': 6, 'I-PRODUCT-COM': 3, 'I-QUANTITY': 2, 'I-QUANTITY-CUR': 3, 'I-QUANTITY-DIM': 3, 'NAN': 2, 'O': 830}


In [32]:
def shuffle(matrix, target, test_proportion):
    ratio = int(matrix.shape[0]/test_proportion)
    X_train = matrix[ratio:,:]
    X_test = matrix[:ratio,:]
    Y_train = target[ratio:]
    Y_test = target[:ratio]
    return X_train, X_test, Y_train, Y_test

In [33]:
X_train, X_test, y_train, y_test = shuffle(X, y, 3)

In [34]:
# def filter_classes(matrix, target, min_class_count = 2):
#     class_counts = Counter(target)
#     classes_to_keep = {cls for cls, count in class_counts.items() if count >= min_class_count}
    
#     mask = np.isin(target, list(classes_to_keep))
    
#     return matrix[mask], target[mask]


In [35]:
# X_train, y_train = filter_classes(X_train, y_train)

In [36]:
# from dask_ml.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [37]:
X_train.shape, y_train.shape

((654368, 69313), (654368,))

In [38]:
X_test.shape, y_test.shape

((327184, 69313), (327184,))

# `Random Forest`

In [44]:
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'max_features': ['auto', 'sqrt']
}

rf = RandomForestClassifier(random_state=42, verbose=1, n_jobs=-1)

grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,
    verbose=10,
    scoring='accuracy'
)

In [45]:
# Fit the grid search
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5; 1/48] START max_depth=10, max_features=auto, min_samples_split=2, n_estimators=100


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


[CV 1/5; 1/48] END max_depth=10, max_features=auto, min_samples_split=2, n_estimators=100;, score=0.868 total time=   0.4s
[CV 2/5; 1/48] START max_depth=10, max_features=auto, min_samples_split=2, n_estimators=100


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished


[CV 2/5; 1/48] END max_depth=10, max_features=auto, min_samples_split=2, n_estimators=100;, score=0.868 total time=   0.4s
[CV 3/5; 1/48] START max_depth=10, max_features=auto, min_samples_split=2, n_estimators=100


...

[CV 4/5; 48/48] END max_depth=None, max_features=sqrt, min_samples_split=10, n_estimators=200;, score=0.860 total time=   0.9s
[CV 5/5; 48/48] START max_depth=None, max_features=sqrt, min_samples_split=10, n_estimators=200


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:    0.7s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


[CV 5/5; 48/48] END max_depth=None, max_features=sqrt, min_samples_split=10, n_estimators=200;, score=0.867 total time=   0.9s


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.3s finished


GridSearchCV(cv=5,
             estimator=RandomForestClassifier(n_jobs=-1, random_state=42,
                                              verbose=1),
             param_grid={'max_depth': [10, 20, 30, None],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200]},
             scoring='accuracy', verbose=10)

In [46]:
grid_search_results = pd.DataFrame(grid_search.cv_results_)[['mean_test_score', 'std_test_score', 'params', 'rank_test_score', 'mean_fit_time']]
grid_search_results

,mean_test_score,std_test_score,params,rank_test_score,mean_fit_time
0,0.866461,0.003024,"{'max_depth': 10, 'max_features': 'auto', 'min...",1,0.322500
1,0.866461,0.003024,"{'max_depth': 10, 'max_features': 'auto', 'min...",1,0.627763
2,0.866461,0.003024,"{'max_depth': 10, 'max_features': 'auto', 'min...",1,0.327498
3,0.866461,0.003024,"{'max_depth': 10, 'max_features': 'auto', 'min...",1,0.627292
4,0.866461,0.003024,"{'max_depth': 10, 'max_features': 'auto', 'min...",1,0.316902
5,0.866461,0.003024,"{'max_depth': 10, 'max_features': 'auto', 'min...",1,0.627294
6,0.866461,0.003024,"{'max_depth': 10, 'max_features': 'sqrt', 'min...",1,0.312180
7,0.866461,0.003024,"{'max_depth': 10, 'max_features': 'sqrt', 'min...",1,0.627117
8,0.866461,0.003024,"{'max_depth': 10, 'max_features': 'sqrt', 'min...",1,0.315749
9,0.866461,0.003024,"{'max_depth': 10, 'max_features': 'sqrt', 'min...",1,0.621190


In [47]:
# Print the best parameters and score
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score:", grid_search.best_score_)

# Get the best model
rf = grid_search.best_estimator_

Best parameters: {'max_depth': 10, 'max_features': 'auto', 'min_samples_split': 2, 'n_estimators': 100}
Best cross-validation score: 0.8664607558139534


In [48]:
y_pred_rf= rf.predict(X_test)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished


In [49]:
accuracy_score(y_test, y_pred_rf)

0.8473520249221184

In [50]:
f1_score(y_test, y_pred_rf, average='weighted', zero_division=0)

0.7773347412438995

In [51]:
print(classification_report(y_pred=y_pred_rf, y_true=y_test, zero_division=0))

                       precision    recall  f1-score   support

                    B       0.00      0.00      0.00         5
            B-ADDRESS       0.00      0.00      0.00        32
           B-DATETIME       0.36      0.23      0.28      1181
      B-DATETIME-DATE       0.36      0.19      0.25       660
 B-DATETIME-DATERANGE       0.33      0.01      0.01       154
  B-DATETIME-DURATION       0.09      0.45      0.15       460
       B-DATETIME-SET       0.00      0.00      0.00        13
      B-DATETIME-TIME       0.09      0.16      0.11       115
 B-DATETIME-TIMERANGE       0.00      0.00      0.00       138
              B-EMAIL       0.00      0.00      0.00        15
              B-EVENT       0.24      0.12      0.16       147
          B-EVENT-CUL       0.00      0.00      0.00        91
     B-EVENT-GAMESHOW       0.39      0.12      0.19        88
      B-EVENT-NATURAL       0.00      0.00      0.00        35
        B-EVENT-SPORT       0.72      0.49      0.59  

In [52]:
f1_score(y_test, y_pred_rf, average='weighted', labels=new_classes, zero_division=0)

0.602364543745410

In [53]:
print(classification_report(y_pred=y_pred_rf, y_true=y_test, labels=new_classes, zero_division=0))

                       precision    recall  f1-score   support

                    B       0.00      0.00      0.00         5
            B-ADDRESS       0.00      0.00      0.00        32
           B-DATETIME       0.36      0.23      0.28      1181
      B-DATETIME-DATE       0.36      0.19      0.25       660
 B-DATETIME-DATERANGE       0.33      0.01      0.01       154
  B-DATETIME-DURATION       0.09      0.45      0.15       460
       B-DATETIME-SET       0.00      0.00      0.00        13
      B-DATETIME-TIME       0.09      0.16      0.11       115
 B-DATETIME-TIMERANGE       0.00      0.00      0.00       138
              B-EMAIL       0.00      0.00      0.00        15
              B-EVENT       0.24      0.12      0.16       147
          B-EVENT-CUL       0.00      0.00      0.00        91
     B-EVENT-GAMESHOW       0.39      0.12      0.19        88
      B-EVENT-NATURAL       0.00      0.00      0.00        35
        B-EVENT-SPORT       0.72      0.49      0.59  

# `Save model`

In [ ]:
import pickle

with open(f'/kaggle/working/random_forest_model.pkl', 'wb') as file:
    pickle.dump(rf, file)